<a href="https://colab.research.google.com/github/kgedney/author-id-project/blob/master/experiment2_CNN_CNNandLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/filtered_data.csv')

In [0]:
import re
df['body_no_urls'] = df.apply(lambda row: re.sub(r"http\S+", "<url>", row['body']), axis=1) 

In [0]:
author_dict = {x: " ".join(y["body_no_urls"].tolist()) for x,y in df.groupby("author")}
df_dict = pd.Series(author_dict).rename_axis(['authors']).reset_index(name='text')

In [0]:
from nltk.tokenize import word_tokenize
df_dict['tokenized_nltk']  = df_dict.apply(lambda row: word_tokenize(row['text']), axis=1)

In [16]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [0]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Conv1D, Dropout, MaxPooling1D, Embedding, GlobalMaxPooling1D, LSTM

In [0]:
df_dict['authors_id'] = pd.Categorical(df_dict.index).codes

In [32]:
x = df_dict['tokenized_nltk'].values
y = df_dict['authors_id'].values

# create sequences
max_features = 20000
tokenizer    = Tokenizer(num_words = max_features)
tokenizer.fit_on_texts(x)
x_sequences  = tokenizer.texts_to_sequences(x)

# pad each sequence to be max length
maxlen = max(len(x) for x in x_sequences)
print(maxlen)
x_sequences = sequence.pad_sequences(x_sequences, maxlen)

73502


In [0]:
x_train, x_test, y_train, y_test = train_test_split(x_sequences, y, test_size=0.20, random_state=22)

In [0]:
model_conv = Sequential()
model_conv.add(Embedding(max_features, output_dim = 30, input_length=73502))
model_conv.add(Dropout(0.2))
model_conv.add(Conv1D(128, 5, activation='relu'))
model_conv.add(MaxPooling1D(4))
model_conv.add(LSTM(100))
#model_conv.add(GlobalMaxPooling1D())

model_conv.add(Dense(100, activation='softmax'))

In [0]:
from keras import optimizers
opt = optimizers.rmsprop(lr=0.001) # speed up optimization
model_conv.compile(optimizer=opt, loss="sparse_categorical_crossentropy", metrics=["acc"])

In [1]:
history1 = model_conv.fit(x_train, y_train,
            batch_size=128,
            epochs=10,
            validation_data=(x_test, y_test))

NameError: ignored